In [57]:
import pandas as pd

df_train = pd.read_csv("./archive/index.csv")

df_train.head()

,path,class_id
0,marvel/0001/001.jpg,1
1,marvel/0001/002.jpg,1
2,marvel/0001/003.jpg,1
3,marvel/0001/004.jpg,1
4,marvel/0001/005.jpg,1


In [58]:
df_test = pd.read_csv("./archive/test.csv")

df_test.head()

,path,class_id
0,test/001.jpg,32
1,test/002.jpg,32
2,test/003.jpg,32
3,test/004.jpg,32
4,test/005.jpg,1


In [59]:
import os
import cv2
import numpy as np

train_images = []
train_labels = []

test_images = []
test_labels = []

for index, row in df_train.iterrows():
    path = row["path"]
    class_id = row["class_id"]
    image = cv2.imread(os.path.join("./archive", path))
    image = cv2.resize(image, (128, 128))

    train_images.append(image)
    train_labels.append(class_id)

for index, row in df_test.iterrows():
    path = row["path"]
    class_id = row["class_id"]
    image = cv2.imread(os.path.join("./archive", path))
    image = cv2.resize(image, (128, 128))

    test_images.append(image)
    test_labels.append(class_id)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

train_images.shape, train_labels.shape, test_images.shape, test_labels.shape


((371, 128, 128, 3), (371,), (79, 128, 128, 3), (79,))

In [60]:
from tensorflow.keras.utils import to_categorical

# Data Preprocessing
train_images = train_images / 255.0
test_images = test_images / 255.0


train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [61]:
# Divide validation set
from sklearn.model_selection import train_test_split

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.15, random_state=42)

In [62]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()

# Conv2D + MaxPooling2D 레이어
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3), padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.3))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(39, activation='softmax'))

/Users/kyuho/Codes/learn-mlflow/MlFlowStarter/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
# Run Model and Log Mlfow
import mlflow
from tensorflow import keras
from mlflow.models import ModelSignature
from mlflow.types import Schema, TensorSpec

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# create a new MLflow experiment
mlflow.set_experiment("Lego Minifigures Classification")

mlflow.set
# mlflow.autolog()

learning_rate = 0.001
batch_size = 32
epochs = 100

with mlflow.start_run():
    adam = keras.optimizers.Adam(learning_rate = learning_rate)

    model.compile(optimizer = adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    history = model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(val_images, val_labels))
    
    # 메트릭을 한 번에 로깅
    for epoch in range(epochs):
        metrics = {
            "train_accuracy": history.history['accuracy'][epoch],
            "val_accuracy": history.history['val_accuracy'][epoch]
        }
        mlflow.log_metrics(metrics, step=epoch)
        
    # 하이퍼파라미터 로깅
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)

    # Input Schema 생성
    
    input_schema = Schema([
        TensorSpec(np.dtype(np.float32), (-1, 39), "input")
    ])

    # Signature 생성
    signature = ModelSignature(inputs=input_schema)

    # 모델 저장
    mlflow.tensorflow.log_model(
        model,
        "cnn_model",
        signature=signature,
    )

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step - accuracy: 0.0294 - loss: 4.5917

5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 382ms/step - accuracy: 0.0282 - loss: 4.5774 - val_accuracy: 0.0357 - val_loss: 3.6658
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step - accuracy: 0.0465 - loss: 3.6271

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 343ms/step - accuracy: 0.0477 - loss: 3.6310 - val_accuracy: 0.0000e+00 - val_loss: 3.6637
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 294ms/step - accuracy: 0.0517 - loss: 3.6546 - val_accuracy: 0.0000e+00 - val_loss: 3.6641
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 295ms/step - accuracy: 0.0326 - loss: 3.6575 - val_accuracy: 0.0000e+00 - val_loss: 3.6648
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 302ms/step - accuracy: 0.0474 - loss: 3.6580 - val_accuracy: 0.0179 - val_loss: 3.6681
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 287ms/step - accuracy: 0.0220 - loss: 3.6521 - val_accuracy: 0.0179 - val_loss: 3.6687
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 291ms/step - accuracy: 0.0764 - loss: 3.6325 - val_accuracy: 0.0179 - val_loss: 3.6783
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step - accuracy: 0.0312 - loss: 3.6700 - val_accuracy: 0.0000e+00 - val_loss: 3.6682
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 285ms/step - accuracy: 0.0430 - loss: 3.6526 - val_accuracy: 0.0000

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 339ms/step - accuracy: 0.0824 - loss: 3.3897 - val_accuracy: 0.0714 - val_loss: 3.6521
Epoch 21/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step - accuracy: 0.1397 - loss: 3.3139

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 338ms/step - accuracy: 0.1397 - loss: 3.3123 - val_accuracy: 0.0714 - val_loss: 3.6470
Epoch 22/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.1566 - loss: 3.2571

...

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 328ms/step - accuracy: 0.1890 - loss: 2.7288 - val_accuracy: 0.1071 - val_loss: 3.5505
Epoch 29/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 288ms/step - accuracy: 0.2523 - loss: 2.6491 - val_accuracy: 0.1607 - val_loss: 3.6458
Epoch 30/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 286ms/step - accuracy: 0.2102 - loss: 2.8007 - val_accuracy: 0.1250 - val_loss: 3.6602
Epoch 31/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 286ms/step - accuracy: 0.1880 - loss: 2.7679 - val_accuracy: 0.1607 - val_loss: 3.6465
Epoch 32/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 290ms/step - accuracy: 0.2394 - loss: 2.6731 - val_accuracy: 0.1786 - val_loss: 3.5725
Epoch 33/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step - accuracy: 0.3232 - loss: 2.4493 - val_accuracy: 0.1429 - val_loss: 3.6702
Epoch 34/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 288ms/step - accuracy: 0.2383 - loss: 2.5886 - val_accuracy: 0.1607 - val_loss: 3.6556
Epoch 35/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 290ms/step - accuracy: 0.2848 - loss: 2.3930 - val_accuracy: 0.1429 - val_lo

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 337ms/step - accuracy: 0.3134 - loss: 2.2331 - val_accuracy: 0.1786 - val_loss: 3.5350
Epoch 43/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.3803 - loss: 2.0687

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 334ms/step - accuracy: 0.3788 - loss: 2.0690 - val_accuracy: 0.1607 - val_loss: 3.4937
Epoch 44/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 295ms/step - accuracy: 0.3859 - loss: 1.9778 - val_accuracy: 0.1964 - val_loss: 3.5824
Epoch 45/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 286ms/step - accuracy: 0.4063 - loss: 1.9134 - val_accuracy: 0.1964 - val_loss: 3.5555
Epoch 46/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 293ms/step - accuracy: 0.4252 - loss: 1.9473 - val_accuracy: 0.1964 - val_loss: 3.6037
Epoch 47/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step - accuracy: 0.3469 - loss: 2.0656 - val_accuracy: 0.1429 - val_loss: 3.6604
Epoch 48/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 306ms/step - accuracy: 0.3993 - loss: 1.8899 - val_accuracy: 0.1964 - val_loss: 3.9269
Epoch 49/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 293ms/step - accuracy: 0.4050 - loss: 1.9287 - val_accuracy: 0.1607 - val_loss: 3.7463
Epoch 50/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 346ms/step - accuracy: 0.4114 - loss: 1.8376 - val_accuracy: 0.1964 - val_lo

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 341ms/step - accuracy: 0.4133 - loss: 1.7246 - val_accuracy: 0.2857 - val_loss: 3.4743
Epoch 58/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 291ms/step - accuracy: 0.4381 - loss: 1.7404 - val_accuracy: 0.2500 - val_loss: 3.5758
Epoch 59/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 291ms/step - accuracy: 0.4727 - loss: 1.7145 - val_accuracy: 0.2500 - val_loss: 3.6997
Epoch 60/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 296ms/step - accuracy: 0.4409 - loss: 1.6492 - val_accuracy: 0.2321 - val_loss: 3.5625
Epoch 61/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step - accuracy: 0.4179 - loss: 1.7475 - val_accuracy: 0.2321 - val_loss: 3.6756
Epoch 62/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 291ms/step - accuracy: 0.4666 - loss: 1.5359 - val_accuracy: 0.2500 - val_loss: 3.9211
Epoch 63/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 295ms/step - accuracy: 0.4921 - loss: 1.5034 - val_accuracy: 0.3036 - val_loss: 3.8565
Epoch 64/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 304ms/step - accuracy: 0.4787 - loss: 1.6112 - val_accuracy: 0.2321 - val_lo

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 335ms/step - accuracy: 0.5043 - loss: 1.3583 - val_accuracy: 0.3393 - val_loss: 3.3296
Epoch 73/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 300ms/step - accuracy: 0.4673 - loss: 1.6049 - val_accuracy: 0.2321 - val_loss: 3.5164
Epoch 74/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step - accuracy: 0.4723 - loss: 1.4772 - val_accuracy: 0.2500 - val_loss: 3.4854
Epoch 75/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step - accuracy: 0.4611 - loss: 1.5391 - val_accuracy: 0.2679 - val_loss: 3.3544
Epoch 76/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 292ms/step - accuracy: 0.5251 - loss: 1.4354 - val_accuracy: 0.3214 - val_loss: 3.5740
Epoch 77/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 310ms/step - accuracy: 0.5242 - loss: 1.4411 - val_accuracy: 0.3036 - val_loss: 3.7199
Epoch 78/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 293ms/step - accuracy: 0.4498 - loss: 1.5115 - val_accuracy: 0.2857 - val_loss: 3.7220
Epoch 79/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 307ms/step - accuracy: 0.5410 - loss: 1.3304 - val_accuracy: 0.3036 - val_lo

In [64]:
import mlflow